<a href="https://colab.research.google.com/github/KenzaB27/TransUnet/blob/main/TransUNet_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Medical Image Segmentation with Vision Transformers as a strong Encoder


In [ ]:
from data_processing.data_parser import DataWriter
from models.transunet import TransUnet
from data_processing.data_parser import DataReader
from experiments.config import get_transunet
import data_processing.data_parser as data_parser
import utils.visualize as visualize
import experiments.config as conf
import importlib
import models.encoder_layers as encoder_layers
import models.decoder_layers as decoder_layers
from models.resnet_v2 import ResNetV2
import numpy as np
import tensorflow as tf

In [ ]:
data_zip = r"train_npz.rar"
tf_record_dir = r"tf_record/"
save_path= r"save_model/"

**Change data format to tfRecord format**
if tfRecord file is availble, you should not run cell below

In [ ]:
dw = DataWriter(src_path=data_zip,dest_path=tf_record_dir,batch_size=25, height=224, width=224)
test_dataset = dw.write_batch_tfrecords()

**Prepare data from tfRecord Data**

In [ ]:
## Prepare data
dr = DataReader(src_path=tf_record_dir, height=224, width=224, depth=3)
training_dataset = dr.get_dataset_training(image_size=224,validation=False)

In [ ]:
## Train Model
config = get_transunet()
transunet = TransUnet(config)
transunet.compile()
training_dataset
transunet.model.summary()

**Train**

In [ ]:
history = transunet.train(training_dataset, save_path, epochs=50)

**Load weights and model**

In [ ]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/Data/savekeras/my_model.h5',
                                       custom_objects={"ResNetV2":ResNetV2,"AddPositionEmbs":encoder_layers.AddPositionEmbs,
                                                       "TransformerBlock":encoder_layers.TransformerBlock,"DecoderCup":decoder_layers.DecoderCup,
                                                       "SegmentationHead":decoder_layers.SegmentationHead,"segmentation_loss":TransUnet.segmentation_loss})

# Show the model architecture
new_model.summary()

**Test**

In [ ]:
dir1 = r'/content/drive/MyDrive/Data/train_npz/Image_87_20.npz'
a=np.load(dir1)
visualize.visualize_non_empty_predictions(a['image'],a['label'],[new_model])